# Dataset.map()

This notebook shows a workflow for using `Dataset.map`. This method is useful for creating a new column with a custom map function to generate the output.


In [1]:
import lilac as ll

ll.set_project_dir('./data')

try:
  glue = ll.get_dataset('local', 'glue_ax_map')
except:
  glue = ll.create_dataset(
    ll.DatasetConfig(
      namespace='local',
      name='glue_ax_map',
      source=ll.HuggingFaceSource(
        dataset_name='glue',
        config_name='ax',
      )))

#ll.start_server()


/Users/nikhil/Code/lilac/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Upper case 'premise'

The following map will upper case the 'premise' field from the dataset.

The output of the map is returned as a generator.


In [2]:
# Upper case 'premise' and print the first result
# This call does not save the output to a column.
res = glue.map(lambda item: item['premise'].upper())
print(next(iter(res)))
print()

# Write the output to a column 'premise_upper'.
glue.map(lambda item: item['premise'].upper(), output_path='premise_upper', overwrite=True)

rows = glue.select_rows(['premise', 'premise_upper'], limit=3)
for row in rows:
  print(row)


Computing map over ['*', '__rowid__']:   0%|          | 0/1104 [00:00<?, ?it/s]

Computing map over ['*', '__rowid__']: 100%|██████████| 1104/1104 [00:00<00:00, 53158.89it/s]


THE CAT SAT ON THE MAT.



Computing map over ['*', '__rowid__']: 100%|██████████| 1104/1104 [00:00<00:00, 57608.48it/s]

Wrote map output to ./data/datasets/local/glue_ax_map/premise_upper-00000-of-00001.parquet
{'premise': 'The cat sat on the mat.', 'premise_upper': 'THE CAT SAT ON THE MAT.'}
{'premise': "When you've got no snow, it's really hard to learn a snow sport so we looked at all the different ways I could mimic being on snow without actually being on snow.", 'premise_upper': "WHEN YOU'VE GOT NO SNOW, IT'S REALLY HARD TO LEARN A SNOW SPORT SO WE LOOKED AT ALL THE DIFFERENT WAYS I COULD MIMIC BEING ON SNOW WITHOUT ACTUALLY BEING ON SNOW."}
{'premise': "When you've got snow, it's really hard to learn a snow sport so we looked at all the different ways I could mimic being on snow without actually being on snow.", 'premise_upper': "WHEN YOU'VE GOT SNOW, IT'S REALLY HARD TO LEARN A SNOW SPORT SO WE LOOKED AT ALL THE DIFFERENT WAYS I COULD MIMIC BEING ON SNOW WITHOUT ACTUALLY BEING ON SNOW."}


# Map continuation during an error, or computer shutdown

`dataset.map()` will not lose data if an error is thrown when writing to disk. The next time it is called, it will continue from where it left off. Once it is finally complete, the column is written.


In [4]:
throw_after_n = 10
i = 0


def _upper(item):
  global i, throw_after_n
  if throw_after_n and i > throw_after_n:
    raise ValueError(f'Throwing after {i}')
  i += 1
  print(item['premise'].upper())
  return item['premise'].upper()


# This is going to throw after 10 iterations. When we call it again, it will only call _upper()
# for the rest of the dataset.
glue.map(_upper, output_path='premise_upper2')

throw_after_n = False
# This will finish calling _upper, without calling it for the first 10 items.
glue.map(_upper, output_path='premise_upper2')


Computing map over ['*', '__rowid__']:   1%|          | 11/1104 [00:00<00:00, 1100.11it/s]

BREXIT IS A REVERSIBLE DECISION, SIR MIKE RAKE, THE CHAIRMAN OF WORLDPAY AND EX-CHAIRMAN OF BT GROUP, SAID AS CALLS FOR A SECOND EU REFERENDUM WERE SPARKED LAST WEEK.
WE BUILT OUR SOCIETY ON UNCLEAN ENERGY.
WE BUILT OUR SOCIETY ON CLEAN ENERGY.
PURSUING A STRATEGY OF NONVIOLENT PROTEST, GANDHI TOOK THE ADMINISTRATION BY SURPRISE AND WON CONCESSIONS FROM THE AUTHORITIES.
PURSUING A STRATEGY OF VIOLENT PROTEST, GANDHI TOOK THE ADMINISTRATION BY SURPRISE AND WON CONCESSIONS FROM THE AUTHORITIES.
PURSUING A STRATEGY OF NONVIOLENT PROTEST, GANDHI TOOK THE ADMINISTRATION BY SURPRISE AND WON CONCESSIONS FROM THE AUTHORITIES.
PURSUING A STRATEGY OF PROTEST, GANDHI TOOK THE ADMINISTRATION BY SURPRISE AND WON CONCESSIONS FROM THE AUTHORITIES.
AND IF BOTH APPLY, THEY ARE ESSENTIALLY IMPOSSIBLE.
AND IF BOTH APPLY, THEY ARE ESSENTIALLY POSSIBLE.
WRITING JAVA IS NOT TOO DIFFERENT FROM PROGRAMMING WITH HANDCUFFS.
WRITING JAVA IS SIMILAR TO PROGRAMMING WITH HANDCUFFS.


ValueError: Throwing after 11